In [1]:
import os
import pickle
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Precision, Recall, F1, and plot confusion matrix libs
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Read CIFAR-10 batch file
def load_cifar10_train(root_dir):
    all_data = []
    all_labels = []
    for i in range(1, 6):
        file_name = os.path.join(root_dir, f"data_batch_{i}")
        with open(file_name, 'rb') as fo:
            data_dict = pickle.load(fo, encoding='bytes')
            # data_dict[b'data'] shape: (10000, 3072)
            # data_dict[b'labels'] length: 10000
            all_data.append(data_dict[b'data'])
            all_labels.append(data_dict[b'labels'])
            
    # 拼接
    all_data = np.concatenate(all_data, axis=0)  # (50000, 3072)
    all_labels = np.concatenate(all_labels, axis=0)  # (50000,)
    
    # 把 numpy 转为 torch.Tensor
    # 先 reshape 成 (N, 3, 32, 32)
    all_data = all_data.reshape(-1, 3, 32, 32).astype(np.float32)
    # 转成 tensor
    X_train = torch.from_numpy(all_data)  # shape: (50000, 3, 32, 32)
    y_train = torch.from_numpy(all_labels).long()  # shape: (50000,)
    
    return X_train, y_train

def load_cifar10_test(root_dir):
    """
    加载 test_batch 文件并返回 (X_test, y_test)
    """
    file_name = os.path.join(root_dir, "test_batch")
    with open(file_name, 'rb') as fo:
        data_dict = pickle.load(fo, encoding='bytes')
        test_data = data_dict[b'data']  # shape: (10000, 3072)
        test_labels = data_dict[b'labels']  # list of length 10000
    
    test_data = test_data.reshape(-1, 3, 32, 32).astype(np.float32)
    X_test = torch.from_numpy(test_data)
    y_test = torch.tensor(test_labels, dtype=torch.long)
    
    return X_test, y_test

In [3]:
# 假设你的数据目录是：
cifar_root = './datasets/cifar-10-batches-py'

# 加载训练与测试
X_train, y_train = load_cifar10_train(cifar_root)  # shape: (50000, 3, 32, 32), (50000,)
X_test, y_test   = load_cifar10_test(cifar_root)   # shape: (10000, 3, 32, 32), (10000,)

# 在 MLP 中，我们通常会 flatten 图像 (3*32*32=3072)。这里可以做两种方法：
# 方法A：在创建 dataset 时就直接 flatten
# 方法B：在 forward 时 flatten
# 这里演示方法A

X_train = X_train.view(X_train.size(0), -1)  # (50000, 3072)
X_test  = X_test.view(X_test.size(0), -1)    # (10000, 3072)

# 用 TensorDataset 直接打包 (data, label)
train_dataset = TensorDataset(X_train, y_train)
test_dataset  = TensorDataset(X_test, y_test)

# 用 DataLoader 来分批迭代
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [4]:
class SimpleMLP(nn.Module):
    def __init__(self, input_dim=3072, hidden_dims=[512, 256, 128], num_classes=10):
        super().__init__()
        layers = []
        prev_dim = input_dim
        for h in hidden_dims:
            layers.append(nn.Linear(prev_dim, h))
            layers.append(nn.ReLU())
            prev_dim = h
        layers.append(nn.Linear(prev_dim, num_classes))
        
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        # x shape: (batch_size, 3072)
        return self.model(x)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [6]:
model = SimpleMLP(input_dim=3072, hidden_dims=[512,256,128], num_classes=10).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 20
for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion)
    test_loss, test_acc   = eval_epoch(model, test_loader, criterion)
    
    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, "
          f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")

Epoch [1/20] Train Loss: 4.1193, Train Acc: 0.2674, Test Loss: 1.9009, Test Acc: 0.3309
Epoch [2/20] Train Loss: 1.7996, Train Acc: 0.3597, Test Loss: 1.7669, Test Acc: 0.3656
Epoch [3/20] Train Loss: 1.7274, Train Acc: 0.3855, Test Loss: 1.7575, Test Acc: 0.3818
Epoch [4/20] Train Loss: 1.6749, Train Acc: 0.4048, Test Loss: 1.6626, Test Acc: 0.4064
Epoch [5/20] Train Loss: 1.6261, Train Acc: 0.4217, Test Loss: 1.6734, Test Acc: 0.4020
Epoch [6/20] Train Loss: 1.6024, Train Acc: 0.4282, Test Loss: 1.6286, Test Acc: 0.4101
Epoch [7/20] Train Loss: 1.5799, Train Acc: 0.4387, Test Loss: 1.5770, Test Acc: 0.4355
Epoch [8/20] Train Loss: 1.5505, Train Acc: 0.4462, Test Loss: 1.6372, Test Acc: 0.4201
Epoch [9/20] Train Loss: 1.5352, Train Acc: 0.4518, Test Loss: 1.5772, Test Acc: 0.4414
Epoch [10/20] Train Loss: 1.5222, Train Acc: 0.4538, Test Loss: 1.5344, Test Acc: 0.4587
Epoch [11/20] Train Loss: 1.5004, Train Acc: 0.4639, Test Loss: 1.5693, Test Acc: 0.4334
Epoch [12/20] Train Loss: 1.49

In [7]:
def compute_metrics(model, loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            all_preds.extend(predicted.cpu().numpy().tolist())
            all_labels.extend(labels.cpu().numpy().tolist())
    
    precision = precision_score(all_labels, all_preds, average='macro')
    recall    = recall_score(all_labels, all_preds, average='macro')
    f1        = f1_score(all_labels, all_preds, average='macro')
    cm        = confusion_matrix(all_labels, all_preds)
    
    return precision, recall, f1, cm

# 使用示例：在最终训练完成后
precision, recall, f1, cm = compute_metrics(model, test_loader)
print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
print("Confusion Matrix:\n", cm)

Precision: 0.4841, Recall: 0.4807, F1: 0.4776
Confusion Matrix:
 [[495  34  81  24  31  26  15  35 227  32]
 [ 31 529   8  31  13  15  11  25 142 195]
 [ 55  31 273  74 189 119 124  75  48  12]
 [ 27  23  79 328  47 243 101  58  53  41]
 [ 43  13  98  54 433  91 132  82  44  10]
 [ 17  13  71 201  65 416  67  75  55  20]
 [  6  16  52 111 121 110 512  23  33  16]
 [ 30  14  53  58 110  74  31 548  35  47]
 [ 64  41  14  30  19  17   8  17 746  44]
 [ 29 136   9  45  16  17  17  50 154 527]]
